# ADCP Assessment 2.2: ADCP in the Sky

At the time that this notebook was created, there was an issue with the ADCP data displayed on the OOI Data Explorer. The y-axis of pcolormesh plots for ADCP time series have depths up to -500 m which puts these measurements "in the Sky". In this notebook, I will compare the datasets available from the Data Explorer ERDDAP server with datasets for the same sensors from OOINet. We hope this will clarify where the issue is in the data pipeline.